In [1]:
from spacy.tokens import DocBin
from pathlib import Path
import json
import random
import spacy

In [2]:
def read_jsonl(path, encoding='utf-8'):
    path = Path(path) if isinstance(path, str) else path
    return [json.loads(line) for line in path.read_text(encoding=encoding).strip().split('\n')]

In [3]:
nlp = spacy.load("fr_core_news_lg")
nlp.disable_pipe("ner")

In [4]:
data = read_jsonl("data/suv/suv_loc.jsonl",encoding='utf-8')

In [5]:
def ent_parser(doc, ent):
    return doc.char_span(ent[0], ent[1], label=ent[2], alignment_mode='contract' )

r = random.randint(0, len(data)) - 1
ent_parser(nlp(data[r]['text']), data[r]["labels"][0])

cervicaux

In [6]:
def doccano_to_spacy(data, path, filter_labels=[]):
    docbin = DocBin()
    for i,item in enumerate(data):
        doc = nlp(item['text'])
        try:
            doc.set_ents([ent_parser(doc, i) for i in item['labels'] if i[2] in filter_labels])
            docbin.add(doc)
        except Exception as e:
            print(e)
            print(f"{item['id']},{i}")
            print([ent_parser(doc, i).__repr__() for i in item['labels']])
            print([item['text'][i[0]:i[1]] for i in item['labels']])
    docbin.to_disk(f"{path}.spacy")

In [7]:
doccano_to_spacy(data,"data",["OBJ_LOC"])

'NoneType' object has no attribute 'label'
4490,305
['None', '8,1', '1,7', 'hauteur de la carène']
['sophagienne', '8,1', '1,7', 'hauteur de la carène']
'NoneType' object has no attribute 'label'
4733,553
['8,3', '6,2', 'None', 'sous-carinaires']
['8,3', '6,2', 'aortique', 'sous-carinaires']
'NoneType' object has no attribute 'label'
5533,1340
['infra et juxtacentimétriques lomboaortiques', 'iliaques communes', '2,8 versus de 3,4', 'rétroaortique', '2', 'None']
['infra et juxtacentimétriques lomboaortiques', 'iliaques communes', '2,8 versus de 3,4', 'rétroaortique', '2', 'atéroaortique']
'NoneType' object has no attribute 'label'
6007,1796
['hilaires bilatérales', 'sous-carénaires', '6,5', 'None']
['hilaires bilatérales', 'sous-carénaires', '6,5', '10R']
'NoneType' object has no attribute 'label'
8874,2698
['sus-claviculaire bilatérale', 'médiastino-hilaire bilatérale', 'infra médiastinale antérieure droite', 'sus-claviculaire gauche', '18', '2', '2,5', 'sus claviculaire/rétropectoral'

In [8]:
def split_data(data, ratio=.3, seed=0):
    data = list(data) 
    random.shuffle(data)
    n = int(len(data) * ratio)
    return data[n:], data[:n]

In [ ]:
exlude_ids=[305,553,874,1340,1796,2698,2839]
random.seed = 0
filtered_data = [item for i,item in enumerate(data) if i not in exlude_ids ]
train, dev = split_data(filtered_data, ratio=.35, seed=0)
test, dev = split_data(dev, ratio=.5, seed=0)
doccano_to_spacy(dev,"dev",["OBJ_LOC"])
doccano_to_spacy(train,"train",["OBJ_LOC"])
doccano_to_spacy(test,"test",["OBJ_LOC"])